# Designing Novel Toehold Sensors

## Steps 
1. Define a specific target sequence
2. Divide the sequence in windows of the same length of the trigger (36nt)
3. For each possible trigger, define a toehold sensor sequence
4. Filter sequences that contain stop codons
5. Evaluate sequence properties using NUPACK and rank them

### Import dependencies

In [1]:
import os
import os.path
import numpy as np
import pandas as pd
import time
import subprocess
from tqdm.notebook import tnrange

### Define library functions

In [2]:
def reversed_complement(sequence):
    mapping = {'A': 'U', 'G': 'C', 'U': 'A', 'C': 'G'}
    sequence_upper = sequence.upper()

    complement = ''
    for c in sequence_upper:
        complement += mapping[c]

    # reverse the sequence
    return complement[::-1]

def split_sequence(sequence, window):
    sequences = []
    limit = len(sequence) - window + 1

    for i in range(0, limit):
        sequences.append(sequence[i:window + i])

    return sequences

def no_stop(sequence):
    stop = ['UAA', 'UAG', 'UGA']

    for i in range(0, len(sequence), 3):
        if sequence[i:i + 3] in stop:
            return False

    return True

def possible_toehold_B(reg, rev):
    
    #to avoid any posterior conflicts please write loops and linkers in RNA form!
    
    loop = 'GGACUUUAGAACAGAGGAGAUAAAGAUG' # The last three nucleotides is AUG, the start codon.
    
    linker = 'ACCUGGCGGCAGCGCAAGAAG'#( from Green paper 2019 se, NOTE THAT IT DOES NOT HAVE THE MOCLO SITE C,  AATG)
    
    #other linkers = "AACCUGGCGGCAGCGCAAGAAGAUGCGUAAA" 
    toeholds = []
    for n in ['A', 'G', 'U', 'C']: 
        if no_stop(reg[0:11] + linker):
            toeholds.append(rev + loop + reg[0:11] + linker)
            return toeholds      
    
    toeholds.append("STOP")
    return toeholds


### Step 1: Define a target sequence
In this case, we use the  5' of the glycoprotein 1 from PVY:

In [3]:
#seq = "GGAGTTTGGGTTATGATGGATGGAGATGAACAAGTCGAATACCCACTGAAACCAATCGTTGAGAATGCAAAACCAACACTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTAGTTCGTAATCTGCGCGATGGAAGTTTGGCTCGCTATGCTTTTGACTTTTATG"
#seq= "TGCAATGGGATAGAGCTGATCTGCCAGAGCACAGATTAGAAGCGATTTGTGCAGCAATGATAGAATCCTGGGGTTATTTTGAGTTAACGCACCAAATCAGGAGATTCTACTCATGGTTGTTGCAACAGCAACCTTTTTCAACGATAGC"
#seq= "GGAGTTTGGGTTATGATGGATGGAGATGAACAAGTCGAATACCCACTGAAACCAATCGTTGAGAATGCAAAACCAACACTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTAGTTCGTAATCTGCGCGATGGAAGTTTGGCTCGCTATGCTTTTGACTTTTATGAAGTTAC"
seq = "gtactgccaactggatccttcgcgggacgtcctttgtttacgtcccgtcggcgctgaatcccgcggacgacccctctcggggccgcttgggactctctcgtccccttctccgtctgccgttccagccgaccacggggcgcacctctctttacgcggtctccccgtctgtgccttctcatctgccggtccgtgtgcacttcgcttcacctctgcacgttgcatggagaccaccgtgaacgcccatcagatcctgcccaaggtcttacataagaggactcttggactcccagcaatgtcaacgaccgaccttgaggcctacttcaaagactgtgtgtttaaggactg"


Convert to RNA and determine the reverse complement.

In [4]:
processed_sequence = seq.upper().replace('T', 'U') #/.replace(' ', '')
rc = reversed_complement(processed_sequence)

### Step 2: Determine 36-nucleotide sub-sequences
To do this, we make all possible triggers for the direct and reversed complementary sequence. 
¿How many sub-sequences of 36 nt do we need to analyze ?

In [5]:
len(split_sequence(rc,36))

310

In [6]:
d_1 = {'Triggers': split_sequence(processed_sequence,36)}
df_1 = pd.DataFrame(data=d_1)
df_1["Sense"]="Direct"

d_2 = {'Triggers': split_sequence(rc,36)}
df_2 = pd.DataFrame(data=d_2)
df_2["Sense"]="Reversed Complement"
frames = [df_1, df_2]
result = pd.concat(frames)
result.head()

,Triggers,Sense
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct
2,ACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUU,Direct
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct


### Step 3: For each trigger, design a toehold sensor

In [7]:
toeholds = [possible_toehold_B(r,reversed_complement(r))[0] for r in result['Triggers']]
result["Toehold Switch"] = toeholds
result.head()

,Triggers,Sense,Toehold Switch
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct,CAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUACGGACUUUAGA...
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct,ACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUAGGACUUUAGA...
2,ACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUU,Direct,STOP
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct,AAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGGACUUUAGA...
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct,UAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGACUUUAGA...


### Step 4: Remove sensors with STOP codons

In [8]:
df = result[result.iloc[:,2] != "STOP"]
df.head()

,Triggers,Sense,Toehold Switch
0,GUACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUG,Direct,CAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUACGGACUUUAGA...
1,UACUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGU,Direct,ACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGUAGGACUUUAGA...
3,CUGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUU,Direct,AAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGGACUUUAGA...
4,UGCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUA,Direct,UAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCAGGACUUUAGA...
5,GCCAACUGGAUCCUUCGCGGGACGUCCUUUGUUUAC,Direct,GUAAACAAAGGACGUCCCGCGAAGGAUCCAGUUGGCGGACUUUAGA...


### Step 5: Evaluate sequence properties using NUPACK 

Some scoring functions from Ma, D, et al. 2018):
- Three-parameter fit (R2 = 0.57):
- Fold change = –71.7 dfull_sensor  – 49.1 dactive_sensor – 22.6 dbinding_site + 54.3

- Four-parameter fit (R2 = 0.60):
- Fold change = –93.2 dfull_sensor – 43.3 dactive_sensor – 22.1 dbinding_site – 9.4 dmin_target + 61.3

Definitions:
- Ensemble Defect: Represents the average number of incorrectly paired nucleotides at equilibrium, evaluated over the ensemble of the complex.
- dfull_sensor:  Ensemble defect for the full toehold switch sequence and structure. 
- dactive_sensor: Ensemble defect was calculated directly from the sequence from the first base of the loop sequence. A completely single-stranded secondary structure was used for assessing design quality for dactive_sensor.
- dbinding_site: Ensemble defect was calculated in an analogous manner using the pairwise binding probabilities of the complete target RNA sequence and specifying a completely single-stranded ideal secondary structure in the binding site region.

#### Library functions
- Calculation of the minimum free energy (MFE) secondary structure of a singular RNA sequence
- NUPACK analysis

In [28]:
def DG(sequence, result_path, wait=0.2):
    file = open('{}pipo.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.close()
    final=[]
    semi_final=[]
    
    cmd = "mfe -T 29 {}pipo".format(result_path)
    subprocess.run(cmd, universal_newlines=True, shell=True)      # Here is the error.

    with open("{}pipo.mfe".format(result_path)) as res:
        for r in res:
            r = r.strip('\n')
            if not r.startswith('%'):
                r = r.split('\t')
                semi_final.append(r)

    #final.append()                

    os.remove("{}pipo.mfe".format(result_path,))
    os.remove("{}pipo.in".format(result_path))
    return (float(semi_final[2][0]))
            
def complex_defect(sequence, secondary, result_path):
    file = open('{}toeh.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.write("{}".format(secondary))
    file.close()

    defect_toeh = 0
    count = 0
    with subprocess.Popen(["complexdefect", "{}toeh".format(result_path)], stdout=subprocess.PIPE) as proc:
        res = (proc.stdout.read()).decode("utf-8").split('\n')
        for l in res:
            count += 1
            if count == 16:
                defect_toeh = float(l)

    os.remove("{}toeh.in".format(result_path))
    return defect_toeh


Activated_THS_Structure="...(((((............)))))..................................."
linear_Structure_25="........................."
secondary_sensor_B = '.........................(((((((((((...(((((............)))))...))))))))))).....................'
secondary_2="((((((((((((((((((((((((((((((((((((...(((((............)))))...................................+))))))))))))))))))))))))))))))))))))"

#### Calculation of the thermodynamic parameters for each design

In [29]:
n = len(df.Triggers)
dfull_sensor   = [complex_defect(df.iloc[i,2], secondary_sensor_B, "Temp/") for i in tnrange(n, desc='dfull_sensor')]
dactive_sensor = [complex_defect(df.iloc[i,2][36::],Activated_THS_Structure,"Temp/") for i in tnrange(n, desc='dactive_sensor')]
dbinding_site  = [complex_defect(df.iloc[i,2][0:25],linear_Structure_25,"Temp/") for i in tnrange(n, desc='dbinding_site')]
Dg_RBS_linker  = [DG(df.iloc[i,2][48:96],"Temp/",1) for i in tnrange(n, desc='Dg_RBS_linker')]

HBox(children=(FloatProgress(value=0.0, description='dfull_sensor', max=536.0, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, description='dactive_sensor', max=536.0, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, description='dbinding_site', max=536.0, style=ProgressStyle(description_width='initial')), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, description='Dg_RBS_linker', max=536.0, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [25]:
score = 54.3 - 71.7*np.array(dfull_sensor) -49.1*np.array(dactive_sensor) -22.6*np.array(dbinding_site)
score_frame = pd.DataFrame(np.stack([dfull_sensor, dactive_sensor, dbinding_site, Dg_RBS_linker, score]).T, columns=['dfull_sensor', 'dactive_sensor', 'dbinding_site', 'Dg_RBS_linker', 'score'])
sorted = df.join(score_frame).sort_values('score', ascending=False)
sorted.head()

,Triggers,Sense,Toehold Switch,dfull_sensor,dactive_sensor,dbinding_site,Dg_RBS_linker,score
73,UCUUAUGUAAGACCUUGGGCAGGAUCUGAUGGGCGU,Reversed Complement,ACGCCCAUCAGAUCCUGCCCAAGGUCUUACAUAAGAGGACUUUAGA...,0.1586,0.4079,0.1164,-13.8,20.26985
73,CUCUCGGGGCCGCUUGGGACUCUCUCGUCCCCUUCU,Direct,AGAAGGGGACGAGAGAGUCCCAAGCGGCCCCGAGAGGGACUUUAGA...,0.1586,0.4079,0.1164,-13.8,20.26985
72,CCUCUCGGGGCCGCUUGGGACUCUCUCGUCCCCUUC,Direct,GAAGGGGACGAGAGAGUCCCAAGCGGCCCCGAGAGGGGACUUUAGA...,0.1839,0.3705,0.1234,-11.6,20.13398
72,CUCUUAUGUAAGACCUUGGGCAGGAUCUGAUGGGCG,Reversed Complement,CGCCCAUCAGAUCCUGCCCAAGGUCUUACAUAAGAGGGACUUUAGA...,0.1839,0.3705,0.1234,-11.6,20.13398
76,UAUGUAAGACCUUGGGCAGGAUCUGAUGGGCGUUCA,Reversed Complement,UGAACGCCCAUCAGAUCCUGCCCAAGGUCUUACAUAGGACUUUAGA...,0.1610,0.3937,0.1725,-8.1,19.52713


## Save the output to a CSV

In [12]:
sorted.to_csv("outputs/Ranked_designs.csv")